In [4]:
import pandas as pd
import re
import os
import zstandard as zstd
import codecs

In [ ]:
def parse_and_filter_txt(txt_path, required_fields=None, max_games=1500000):
    """
    Parse un fichier TXT en un DataFrame Pandas avec filtrage des champs obligatoires et limite de parties.

    :param txt_path: Chemin du fichier TXT.
    :param required_fields: Liste des champs requis pour garder une partie.
    :param max_games: Nombre maximum de parties à ajouter au DataFrame.
    :return: DataFrame Pandas contenant les parties valides.
    """
    if required_fields is None:
        required_fields = ['WhiteElo', 'BlackElo', 'Opening', 'TimeControl', 'Termination']

    game_data = []  # Liste pour stocker les parties valides

    with open(txt_path, 'r', encoding='utf-8') as f:
        txt_content = f.read()

    # Séparer les parties en utilisant une ligne vide comme séparateur
    games = txt_content.strip().split("\n\n")

    for game in games:
        # Diviser les métadonnées et les coups
        lines = game.split("\n")
        metadata_lines = [line for line in lines if line.startswith("[")]
        moves_lines = [line for line in lines if not line.startswith("[")]

        # Extraire les métadonnées
        metadata = dict(re.findall(r'\[(\w+) "(.*?)"\]', "\n".join(metadata_lines)))

        # Vérifier que tous les champs obligatoires sont présents
        if all(field in metadata for field in required_fields):
            # Extraire les coups
            moves = " ".join(moves_lines).replace('\n', ' ')  # Joindre les coups en une seule chaîne

            # Ajouter la partie si elle satisfait les critères
            metadata['Moves'] = moves
            game_data.append(metadata)

        # Vérifier si nous avons atteint le nombre maximal de parties
        if len(game_data) >= max_games:
            break

    # Créer un DataFrame à partir des parties valides
    df = pd.DataFrame(game_data)
    return df

# Exemple d'utilisation
txt_file = "/home/onyxia/work/data/Data.txt"
required_fields = ['WhiteElo', 'BlackElo', 'Opening', 'TimeControl', 'Termination']
df = parse_and_filter_txt(txt_file, required_fields=required_fields, max_games=500000)



# Sauvegarder correctement le DataFrame en CSV
output_csv = "Data_uncleaned.csv"
df.to_csv(output_csv, index=False, encoding='utf-8-sig')

Nombre de parties valides détectées : 500000


In [12]:
print(df.shape)

(500000, 18)


In [ ]:
def parse_and_filter_pgn(pgn_path, required_fields=None):
    """
    Parse un fichier PGN en un DataFrame Pandas avec gestion des erreurs et filtrage des champs obligatoires.

    :param pgn_path: Chemin du fichier PGN.
    :param required_fields: Liste des champs requis pour garder une partie.
    :return: DataFrame Pandas contenant les parties valides.
    """
    if required_fields is None:
        required_fields = ['WhiteElo', 'BlackElo', 'Opening', 'TimeControl', 'Termination']

    game_data = []  # Liste pour stocker les parties valides
    invalid_games = []  # Parties non valides pour diagnostic

    with open(pgn_path, 'r', encoding='utf-8') as f:
        pgn_content = f.read()

    # Séparer les parties en utilisant [Event "..."] comme séparateur
    games = re.split(r'\n(?=\[Event\s")', pgn_content.strip())

    for i, game in enumerate(games):
        try:
            # Extraire les métadonnées
            metadata = dict(re.findall(r'\[(\w+) "(.*?)"\]', game))

            # Vérifier que tous les champs obligatoires sont présents
            missing_fields = [field for field in required_fields if field not in metadata]
            if missing_fields:
                raise ValueError(f"Champs obligatoires manquants : {', '.join(missing_fields)}")

            # Extraire les mouvements (facultatif)
            moves = re.search(r'(1\..+)', game, re.DOTALL)
            metadata['Moves'] = moves.group(1).replace('\n', ' ') if moves else ''

            # Ajouter la partie si elle satisfait les critères
            game_data.append(metadata)

        except Exception as e:
            # Enregistrer la partie non valide pour diagnostic
            invalid_games.append((i + 1, str(e), game))

    # Créer un DataFrame à partir des parties valides
    df = pd.DataFrame(game_data)

    # Afficher les erreurs détectées
    if invalid_games:
        print(f"{len(invalid_games)} parties ont été ignorées en raison d'erreurs ou de champs manquants.")
        for idx, (game_no, error, game_content) in enumerate(invalid_games[:5]):  # Limiter à 5 erreurs
            print(f"Partie {game_no} ignorée : {error}\nContenu :\n{game_content}\n")

    return df

# Exemple d'utilisation
pgn_file = "/home/onyxia/work/data/output.txt"
required_fields = ['WhiteElo', 'BlackElo', 'Opening', 'TimeControl', 'Termination']
df = parse_and_filter_pgn(pgn_file, required_fields=required_fields)

# Assurez-vous que le DataFrame est bien créé
print(f"Nombre de parties valides détectées : {len(df)}")

# Sauvegarder correctement le DataFrame en CSV
output_csv = "Data_uncleaned.csv"
df.to_csv(output_csv, index=False, encoding='utf-8-sig')
print(f"Fichier CSV écrit avec succès : {output_csv}")

In [ ]:
import s3fs
import os

# Chemin du fichier CSV local
local_file_path = "/home/onyxia/work/data/Data_uncleaned.csv"
local_file_path2 = "/home/onyxia/work/data/output.txt"
# Configuration du bucket S3 et du chemin de destination
MY_BUCKET = "charlret"
FILE_PATH_OUT_S3 = f"{MY_BUCKET}/diffusion/Data.csv"
FILE_PATH_OUT_S3_2 = f"{MY_BUCKET}/diffusion/texte.txt"


os.environ["AWS_ACCESS_KEY_ID"] = 'FVWK0NHMH7H731QHWURF'
os.environ["AWS_SECRET_ACCESS_KEY"] = 'RJ5oGkNswBPVxH76iVt2dwizr5+A7HiKEANj0lC7'
os.environ["AWS_SESSION_TOKEN"] = 'eyJhbGciOiJIUzUxMiIsInR5cCI6IkpXVCJ9.eyJhY2Nlc3NLZXkiOiJGVldLME5ITUg3SDczMVFIV1VSRiIsImFsbG93ZWQtb3JpZ2lucyI6WyIqIl0sImF1ZCI6WyJtaW5pby1kYXRhbm9kZSIsIm9ueXhpYSIsImFjY291bnQiXSwiYXV0aF90aW1lIjoxNzM0NTEyMjk5LCJhenAiOiJvbnl4aWEiLCJlbWFpbCI6ImNoYXJsZXMuZGVyYWluQGVuc2FlLmZyIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImV4cCI6MTczNTkwNDk5MCwiZmFtaWx5X25hbWUiOiJEZXJhaW4iLCJnaXZlbl9uYW1lIjoiQ2hhcmxlcyIsImdyb3VwcyI6WyJVU0VSX09OWVhJQSJdLCJpYXQiOjE3MzUzMDAxODgsImlzcyI6Imh0dHBzOi8vYXV0aC5sYWIuc3NwY2xvdWQuZnIvYXV0aC9yZWFsbXMvc3NwY2xvdWQiLCJqdGkiOiIyYTgxNWU1Ny0zZDEyLTRjOGQtOGM2My04Njk3NWU1Nzc3YjIiLCJuYW1lIjoiQ2hhcmxlcyBEZXJhaW4iLCJwb2xpY3kiOiJzdHNvbmx5IiwicHJlZmVycmVkX3VzZXJuYW1lIjoiY2hhcmxyZXQiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImRlZmF1bHQtcm9sZXMtc3NwY2xvdWQiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50IiwibWFuYWdlLWFjY291bnQtbGlua3MiLCJ2aWV3LXByb2ZpbGUiXX19LCJyb2xlcyI6WyJvZmZsaW5lX2FjY2VzcyIsInVtYV9hdXRob3JpemF0aW9uIiwiZGVmYXVsdC1yb2xlcy1zc3BjbG91ZCJdLCJzY29wZSI6Im9wZW5pZCBwcm9maWxlIGdyb3VwcyBlbWFpbCIsInNpZCI6ImRmYzMwNjZkLWFjZjEtNDFmZi04ODU2LWUzNjlkMzFiZjk5OCIsInN1YiI6Ijg3MGIwZWRjLTE2ZGQtNDRmOS1hMDRjLWMyNzNiMzcyOGYyNiIsInR5cCI6IkJlYXJlciJ9.fZwXn4U5ak5NCLCz1wZcPNAqwsYcw7GqGavNIVEMUvtEGeg2uIkoO4Zv7COnNtkWsprn0KtGgj9EU7tMoSbqGA'
os.environ["AWS_DEFAULT_REGION"] = 'us-east-1'
fs = s3fs.S3FileSystem(
    client_kwargs={'endpoint_url': 'https://'+'minio.lab.sspcloud.fr'},
    key = os.environ["AWS_ACCESS_KEY_ID"], 
    secret = os.environ["AWS_SECRET_ACCESS_KEY"], 
    token = os.environ["AWS_SESSION_TOKEN"])

# Ouvrir et envoyer le fichier CSV local vers S3
with open(local_file_path, "rb") as local_file:
    with fs.open(FILE_PATH_OUT_S3, "wb") as s3_file:
        s3_file.write(local_file.read())

with open(local_file_path2, "rb") as local_file:
    with fs.open(FILE_PATH_OUT_S3_2, "wb") as s3_file:
        s3_file.write(local_file.read())

print(f"Fichier '{local_file_path}' a été sauvegardé dans S3 sous '{FILE_PATH_OUT_S3}'")